# Over and Under

## Table of Contents

1. <a href="#item1">Environment Setup</a>
    
2. <a href="#item3">Pull Crime Data from Chicago Data Portal Using API</a> 
    
3. <a href="#item4">Pull Dessert Restaurant Data from the Foursquare API</a> 

4. <a href="#item5">Create Chicago Map</a> 

5. <a href="#item6">Pull Toronto Cannabis Store Data from Google Places API</a> 

6. <a href="#item7">Pull Toronto Dessert Restaurant Data from FourSquare API</a> 

6. <a href="#item7">Create Toronto Map</a> 

7. <a href="#item9">Difference for Chicago Vs Toronto</a> 

<a id="item1"></a>
# Environment Setup

In [1]:
# !pip install ipython-sql
# !pip install ibm_db 
# !pip install ibm_db_sa
# !pip install sodapy

In [2]:
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine
%load_ext sql
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Load config file
import json
with open('/Users/matthewtavares/Google Drive/Coursera/IBM Data Analyst/Coursera_Capstone/Config/config.json') as config_file:
    config_file = json.load(config_file)
    print('File loaded.')

File loaded.


<a id="item3"></a>
# Pull data from Chicago Data Portal using API

In [4]:
from sodapy import Socrata

cdb_token=config_file['cdb_token']
cdb_username=config_file['cdb_username']
cdb_password=config_file['cdb_password']

client = Socrata("data.cityofchicago.org", cdb_token, username=cdb_username, password=cdb_password)

# A quick review of the source data on the Chicago Data Portal revealed the data required has the attribute 'Narcotics' in the 'primary_type' feature.
results = client.get("ijzp-q8t2", where="primary_type = 'NARCOTICS' AND year>2014", limit=100000000)

# Convert to pandas DataFrame
df = pd.DataFrame.from_records(results)

In [5]:
# The columns are collapsed, hiding some, updating options to display all columns
pd.set_option('display.max_columns', None)
df.head()

,id,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,beat,district,ward,community_area,fbi_code,year,updated_on,x_coordinate,y_coordinate,latitude,longitude,location
0,11439193,JB330583,2018-07-01T11:59:00.000,0000X N HOMAN BLVD,2017,NARCOTICS,MANU/DELIVER:CRACK,ALLEY,True,False,1123,011,28,27,18,2018,2019-03-26T16:15:20.000,NaN,NaN,NaN,NaN,NaN
1,11478282,JB469822,2018-10-10T10:00:00.000,006XX W OHARE ST,1812,NARCOTICS,POSS: CANNABIS MORE THAN 30GMS,GOVERNMENT BUILDING/PROPERTY,True,False,1654,016,41,76,18,2018,2019-03-26T16:15:20.000,NaN,NaN,NaN,NaN,NaN
2,11441749,JB276205,2018-05-24T06:00:00.000,012XX W WILSON AVE,2050,NARCOTICS,CRIMINAL DRUG CONSPIRACY,STREET,True,False,1913,019,46,3,18,2018,2019-03-26T16:15:20.000,NaN,NaN,NaN,NaN,NaN
3,11478264,JB469907,2018-10-10T11:25:00.000,033XX W FILLMORE ST,2093,NARCOTICS,FOUND SUSPECT NARCOTICS,POLICE FACILITY/VEH PARKING LOT,True,False,1134,011,24,29,18,2018,2019-03-26T16:15:20.000,NaN,NaN,NaN,NaN,NaN
4,11478271,JB469876,2018-10-10T11:00:00.000,033XX W FILLMORE ST,2093,NARCOTICS,FOUND SUSPECT NARCOTICS,POLICE FACILITY/VEH PARKING LOT,True,False,1134,011,24,29,18,2018,2019-03-26T16:15:20.000,NaN,NaN,NaN,NaN,NaN


In [6]:
# Check shape of table
df.shape

(80748, 22)

## Cleanup and analyze

In [7]:
# Remove any unrequired features
df.drop(columns=['case_number', 
                 'block', 
                 'iucr', 
                 'arrest', 
                 'domestic', 
                 'location_description',
                 'ward',
                 'community_area',
                 'beat', 
                 'fbi_code', 
                 'x_coordinate', 
                 'y_coordinate', 
                 'updated_on', 
                 'location'], inplace=True)

In [8]:
# Remove NAN rows from 'latitude' and 'longitude' as we can't map them
df.dropna(subset=['latitude', 'longitude'], inplace=True)
df.reset_index(inplace=True, drop=True)

In [9]:
# Check the shape of df after clean up
df.shape

(77457, 8)

In [10]:
df.dtypes

id              object
date            object
primary_type    object
description     object
district        object
year            object
latitude        object
longitude       object
dtype: object

In [11]:
# Correct datatypes for columns
convert_dict=({'id': 'int64', 
           'date':'datetime64', 
           'primary_type': 'str', 
           'description': 'str',
           'district': 'str',
           'latitude': 'float64',
           'longitude': 'float64',})

df = df.astype(convert_dict) 
print(df.dtypes) 

id                       int64
date            datetime64[ns]
primary_type            object
description             object
district                object
year                    object
latitude               float64
longitude              float64
dtype: object


In [12]:
# All the rows related to cannabis contain 'CANNABIS' in the feature 'description'. Keeping these rows
df=df[df['description'].str.contains("CANNABIS")]
df

,id,date,primary_type,description,district,year,latitude,longitude
4,11706747,2019-05-31 18:13:00,NARCOTICS,POSS: CANNABIS 30GMS OR LESS,024,2019,42.014720,-87.702914
7,11712396,2019-06-05 19:27:00,NARCOTICS,MANU/DEL:CANNABIS OVER 10 GMS,005,2019,41.671419,-87.642913
11,11695112,2019-05-21 20:52:00,NARCOTICS,POSS: CANNABIS 30GMS OR LESS,004,2019,41.747393,-87.585439
13,11694518,2019-05-21 11:35:00,NARCOTICS,MANU/DEL:CANNABIS OVER 10 GMS,006,2019,41.740780,-87.646186
14,11694558,2019-05-21 12:00:00,NARCOTICS,MANU/DEL:CANNABIS OVER 10 GMS,025,2019,41.903148,-87.756465
...,...,...,...,...,...,...,...,...
77451,9999947,2015-03-18 19:15:00,NARCOTICS,POSS: CANNABIS MORE THAN 30GMS,002,2015,41.794004,-87.622449
77452,9999950,2015-03-18 20:32:00,NARCOTICS,POSS: CANNABIS 30GMS OR LESS,005,2015,41.685452,-87.610410
77453,9999954,2015-03-18 19:51:00,NARCOTICS,POSS: CANNABIS 30GMS OR LESS,011,2015,41.897016,-87.737314
77455,9999969,2015-03-18 20:30:00,NARCOTICS,POSS: CANNABIS 30GMS OR LESS,011,2015,41.881893,-87.738538


In [13]:
# Checking the descriptions and volume of each one
df.groupby('description').count()

,id,date,primary_type,district,year,latitude,longitude
description,,,,,,,
ATTEMPT POSSESSION CANNABIS,3,3,3,3,3,3,3
CANNABIS PLANT,25,25,25,25,25,25,25
DELIVER CANNABIS TO PERSON <18,11,11,11,11,11,11,11
MANU/DEL:CANNABIS 10GM OR LESS,2873,2873,2873,2873,2873,2873,2873
MANU/DEL:CANNABIS OVER 10 GMS,3741,3741,3741,3741,3741,3741,3741
MANUFACTURE / DELIVER - CANNABIS 10 GRAMS OR LESS,16,16,16,16,16,16,16
MANUFACTURE / DELIVER - CANNABIS OVER 10 GRAMS,124,124,124,124,124,124,124
POSS: CANNABIS 30GMS OR LESS,17767,17767,17767,17767,17767,17767,17767
POSS: CANNABIS MORE THAN 30GMS,3927,3927,3927,3927,3927,3927,3927


In [14]:
# There are a few descriptions that only have minor syntax differences. Going to clean these up
df.replace(to_replace="POSSESS - CANNABIS 30 GRAMS OR LESS", value="POSS: CANNABIS 30GMS OR LESS", inplace=True)
df.replace(to_replace="POSSESS - CANNABIS MORE THAN 30 GRAMS", value="POSS: CANNABIS MORE THAN 30GMS", inplace=True)
df.replace(to_replace="MANUFACTURE / DELIVER - CANNABIS 10 GRAMS OR LESS", value="MANU/DEL:CANNABIS 10GM OR LESS", inplace=True)
df.replace(to_replace="MANUFACTURE / DELIVER - CANNABIS OVER 10 GRAMS", value="MANU/DEL:CANNABIS OVER 10 GMS", inplace=True)

In [15]:
# Drop rows with 'Cannabis Plant' and 'Deliver Cannabis to Person <18' and 'attempt possession cannabis' theses are superfluous values 
# that would charged conncurrently with other possesion charges
df.drop(df[df['description'] == 'CANNABIS PLANT'].index , inplace=True)
df.drop(df[df['description'] == 'DELIVER CANNABIS TO PERSON <18'].index , inplace=True)
df.drop(df[df['description'] == 'ATTEMPT POSSESSION CANNABIS'].index , inplace=True)

In [16]:
df.groupby('description').count()

,id,date,primary_type,district,year,latitude,longitude
description,,,,,,,
MANU/DEL:CANNABIS 10GM OR LESS,2889,2889,2889,2889,2889,2889,2889
MANU/DEL:CANNABIS OVER 10 GMS,3865,3865,3865,3865,3865,3865,3865
POSS: CANNABIS 30GMS OR LESS,17831,17831,17831,17831,17831,17831,17831
POSS: CANNABIS MORE THAN 30GMS,4071,4071,4071,4071,4071,4071,4071


In [17]:
df.shape

(28656, 8)

In [18]:
# Create new category features for distribution/possession
chi_dist = df.loc[(df['description']=='MANU/DEL:CANNABIS 10GM OR LESS') | (df['description']=='MANU/DEL:CANNABIS OVER 10 GMS')]
chi_buy = df.loc[(df['description']=='POSS: CANNABIS 30GMS OR LESS') | (df['description']=='POSS: CANNABIS MORE THAN 30GMS')]

## Save to CSV

In [19]:
compression_opts = dict(method='zip',archive_name='data/chi_buy.csv')  
chi_buy.to_csv('data/chi_buy.zip', index=False, compression=compression_opts)

compression_opts = dict(method='zip',archive_name='data/chi_dist.csv')  
chi_dist.to_csv('data/chi_dist.zip', index=False, compression=compression_opts)

<a id="item4"></a>
# Pull restaurant data from Foursquare API

In [20]:
# Import tools to manage interact with FourSquare API and data
import requests
from pandas.io.json import json_normalize

In [21]:
# Setup credential for FourSquare API and OAuth
client_id = config_file['fs_id']
client_secret = config_file['fs_secret']
version = '20180604'
limit = 1000

In [ ]:
# Call oath url from notebook and open in browser
import webbrowser

rdurl='http://www.google.com/'
url='https://foursquare.com/oauth2/authenticate?client_id={}&response_type=code&redirect_uri={}'.format(client_id, rdurl)

webbrowser.open(url)

In [ ]:
# Enter oauth code from url below
oauth='4HSETHKPSNKYD2P5BFMVRV3ZBXQ5F2DR0LWW1O4WAEPUVH4O#_=_'

# Call for token
url='https://foursquare.com/oauth2/access_token?client_id={}&client_secret={}&grant_type=authorization_code&redirect_uri={}&code={}'.format(client_id,client_secret, rdurl, oauth)
webbrowser.open(url)

In [22]:
# Access Token
oauth_token='4TZIVH0DBGKZMSZ1YNPVUWLLRKN1RIN45GW1LYSRGR5GM4VE'

In [23]:
# Request for venues from FourSquare API
search_query = 'Dessert'
near = 'Chicago'
radius= 30000
url = 'https://api.foursquare.com/v2/venues/search?radius={}&query={}&near={}&v={}&limit={}&oauth_token={}'.format(radius, search_query, near, version, limit, oauth_token)
results = requests.get(url).json()

In [24]:
# Create dict with relevant data from JSON results
fs_venues = results['response']['venues']

# Convert venues to df
fs_venues = json_normalize(fs_venues)
fs_venues.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id
0,543b2c66498ed351148bccd4,Dessert De Mango,"[{'id': '4bf58dd8d48988d1d0941735', 'name': 'D...",v-1594736341,False,2516 S Halsted St,Halsted & Archer,41.846649,-87.646591,"[{'label': 'entrance', 'lat': 41.846546, 'lng'...",60608,US,Chicago,IL,United States,"[2516 S Halsted St (Halsted & Archer), Chicago...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5d576c1144209c0008ffac5c,Mango Mango Dessert,"[{'id': '4bf58dd8d48988d1d0941735', 'name': 'D...",v-1594736341,False,NaN,NaN,41.853546,-87.634690,"[{'label': 'display', 'lat': 41.853546, 'lng':...",60616,US,Chicago,IL,United States,"[Chicago, IL 60616, United States]",1947268,https://www.grubhub.com/restaurant/mango-mango...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,NaN
2,5cdebb2dfc9e94002c02f17a,Relo's Board Game & Dessert Cafe,"[{'id': '4bf58dd8d48988d1d0941735', 'name': 'D...",v-1594736341,False,1321 W Taylor St,NaN,41.869213,-87.660108,"[{'label': 'display', 'lat': 41.86921328706415...",60607,US,Chicago,IL,United States,"[1321 W Taylor St, Chicago, IL 60607, United S...",1541405,https://www.grubhub.com/restaurant/relos-board...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,NaN
3,5ade6ed52be42569a1386588,Dessert Dealer,"[{'id': '4bf58dd8d48988d1d0941735', 'name': 'D...",v-1594736341,False,NaN,NaN,41.883411,-87.648640,"[{'label': 'display', 'lat': 41.883411, 'lng':...",60607,US,Chicago,IL,United States,"[Chicago, IL 60607, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,538b756a498e1b0d8c1b0169,No Dessert Left Behind Event,"[{'id': '4bf58dd8d48988d1d0941735', 'name': 'D...",v-1594736341,False,1520 W Fulton Market,NaN,41.878110,-87.665089,"[{'label': 'display', 'lat': 41.87810998022422...",60607,US,Chicago,IL,United States,"[1520 W Fulton Market, Chicago, IL 60607, Unit...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
fs_venues.shape

(24, 23)

## Clean data

In [26]:
# Remove any unrequired features
fs_venues.drop(columns=['id',
                        'categories',
                        'referralId',
                        'hasPerk',
                        'location.crossStreet',
                        'location.labeledLatLngs',
                        'location.postalCode',
                        'location.cc',
                        'location.country',
                        'location.formattedAddress',
                        'delivery.id',
                        'delivery.url',
                        'delivery.provider.name',
                        'delivery.provider.icon.prefix',
                        'delivery.provider.icon.sizes',
                        'delivery.provider.icon.name',
                        'venuePage.id'], inplace=True)

In [27]:
# Check datatypes for df
fs_venues.dtypes

name                 object
location.address     object
location.lat        float64
location.lng        float64
location.city        object
location.state       object
dtype: object

## Save to CSV

In [28]:
compression_opts = dict(method='zip',archive_name='data/fs_venues.csv')  
fs_venues.to_csv('data/fs_venues.zip', index=False, compression=compression_opts)

<a id="item5"></a>
# Create the Chicago Map
Add in restaurants from FourSquare and layer on top of cannabis charges

In [29]:
# Import map tools
# !conda install -c conda-forge folium --yes
import folium
from folium import plugins
from folium.plugins import HeatMap

In [30]:
# Center map based on mean of long lat of possesion charges
latitude=chi_buy[['latitude']].mean()
longitude=chi_buy[['longitude']].mean()

# Create map
chi_map = folium.Map(location=[latitude, longitude], zoom_start=11, tiles='cartodbpositron')

feature_heat_buy = folium.FeatureGroup(name='Heat Map - Cannabis Possesion Charges')
feature_rest = folium.FeatureGroup(name='Dessert Restaurants')

HeatMap(chi_buy[['latitude','longitude']], radius=15).add_to(feature_heat_buy)

for i, v in fs_venues.iterrows():
    popup = """
    Restaurant Name : <b>%s</b><br>
    """ % (v['name'])
    folium.Marker(location=[v['location.lat'], v['location.lng']],
    radius=3,
    tooltip=popup,
    icon=folium.Icon(color='blue', icon='fa-foursquare', prefix='fa')
    ).add_to(feature_rest)

folium.CircleMarker(location=[41.880, -87.735], radius=60, popup='Circle 1', color='#3186cc', fill=True, fill_color='#3186cc').add_to(chi_map)
folium.CircleMarker(location=[41.880, -87.735], radius=2, popup='Circle 1', color='#3186cc', fill=True, fill_color='#3186cc').add_to(chi_map)

folium.CircleMarker(location=[41.770, -87.67], radius=40, popup='Circle 1', color='#3186cc', fill=True, fill_color='#3186cc').add_to(chi_map)
folium.CircleMarker(location=[41.770, -87.67], radius=2, popup='Circle 1', color='#3186cc', fill=True, fill_color='#3186cc').add_to(chi_map)

folium.CircleMarker(location=[41.762, -87.567], radius=30, popup='Circle 1', color='#3186cc', fill=True, fill_color='#3186cc').add_to(chi_map)
folium.CircleMarker(location=[41.762, -87.567], radius=2, popup='Circle 1', color='#3186cc', fill=True, fill_color='#3186cc').add_to(chi_map)

feature_heat_buy.add_to(chi_map)
feature_rest.add_to(chi_map)

folium.LayerControl(collapsed=False).add_to(chi_map)

# Display map
chi_map

In [136]:
# Determine distance of the most dense possesion areas in red to the closest dessert restaurant
import geopy.distance

coords_1 = (41.880, -87.735)
coords_2 = (float(fs_venues[fs_venues['name']=='Lick Her Desserts'].iloc[:,2]),float(fs_venues[fs_venues['name']=='Lick Her Desserts'].iloc[:,3]))
chi_distance1 = geopy.distance.distance(coords_1, coords_2).km

coords_1 = (41.770, -87.67)
coords_2 = (float(fs_venues[fs_venues['name']=='Five Star Dessert'].iloc[:,2]),float(fs_venues[fs_venues['name']=='Five Star Dessert'].iloc[:,3]))
chi_distance2 = geopy.distance.distance(coords_1, coords_2).km

coords_1 = (41.762, -87.567)
coords_2 = (float(fs_venues[fs_venues['name']=='Five Star Dessert'].iloc[:,2]),float(fs_venues[fs_venues['name']=='Five Star Dessert'].iloc[:,3]))
chi_distance3 = geopy.distance.distance(coords_1, coords_2).km

print('Circle 1 Distance: {:.2f}.km\nCircle 2 Distance: {:.2f}.km\nCircle 3 Distance: {:.2f}.km'.format(munchie_distance1, munchie_distance2, munchie_distance3))

Circle 1 Distance: 0.23.km
Circle 2 Distance: 0.43.km
Circle 3 Distance: 0.09.km


In [137]:
chi_average_distance = (chi_distance1+chi_distance2+chi_distance3)/3
chi_average_distance

5.2623749500281

<a id="item6"></a>
# Get Data from Google Place API
Foursquare doesnt have a lot of cannabis stores in Toronto in it's DB, therefore using the Gooogle Places API
This is going to be a complex because the places API only returns 20 locations at time and will on take 3 requests for a total 60 locations.

In [35]:
g_api_key=config_file['google_api']

In [36]:
# Request #1 criteria
radius = 5000
keyword = 'Cannabis Store'
to_lat = 43.6532
to_long = -79.3832

In [45]:
# Request 1
url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={},{}&radius={}&keyword={}&key={}'.format(to_lat, to_long, radius, keyword,g_api_key)
results = requests.get(url).json()
# Convert venues to df
gvenues = json_normalize(results['results'])

In [46]:
# Request 2 using next page token
npt=results['next_page_token']
url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?pagetoken={}&key={}'.format(npt, g_api_key)
results2 = requests.get(url).json()
# Convert venues to df
gvenues2 = json_normalize(results2['results'])

In [47]:
# Request 3
npt=results2['next_page_token']
url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?pagetoken={}&key={}'.format(npt, g_api_key)
results3 = requests.get(url).json()
# Convert venues to df
gvenues3 = json_normalize(results3['results'])

In [48]:
# Merge the 3 DFs
g_to_cannabis_venues = gvenues.append(gvenues2, ignore_index=True)
g_to_cannabis_venues = g_to_cannabis_venues.append(gvenues3, ignore_index=True)

In [49]:
g_to_cannabis_venues.head(2)

,business_status,icon,id,name,photos,place_id,rating,reference,scope,types,user_ratings_total,vicinity,geometry.location.lat,geometry.location.lng,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,opening_hours.open_now,plus_code.compound_code,plus_code.global_code,price_level
0,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,d2573e4fbd164207b2c85dba55ba9355a213d6a9,NOVA Cannabis Queen Street,"[{'height': 1600, 'html_attributions': ['<a hr...",ChIJ_Xw2-8I1K4gR7Ph8G3KDJ2M,4.1,ChIJ_Xw2-8I1K4gR7Ph8G3KDJ2M,GOOGLE,"[point_of_interest, store, establishment]",460,"499 Queen St W, Toronto",43.648110,-79.398372,43.649562,-79.397063,43.646862,-79.399762,False,"JJX2+6M Toronto, Ontario",87M2JJX2+6M,NaN
1,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,b477de8adca9dcad386d41e9f70fec9ebcccb224,Canna Cabana - Yonge Street,"[{'height': 4032, 'html_attributions': ['<a hr...",ChIJs6sGC2U1K4gRh1079uU0s6A,4.2,ChIJs6sGC2U1K4gRh1079uU0s6A,GOOGLE,"[point_of_interest, store, establishment]",281,"435B Yonge St, Toronto",43.660918,-79.382577,43.662495,-79.381295,43.659795,-79.383995,True,"MJ68+9X Toronto, Ontario",87M2MJ68+9X,NaN


In [50]:
list(g_to_cannabis_venues)

['business_status',
 'icon',
 'id',
 'name',
 'photos',
 'place_id',
 'rating',
 'reference',
 'scope',
 'types',
 'user_ratings_total',
 'vicinity',
 'geometry.location.lat',
 'geometry.location.lng',
 'geometry.viewport.northeast.lat',
 'geometry.viewport.northeast.lng',
 'geometry.viewport.southwest.lat',
 'geometry.viewport.southwest.lng',
 'opening_hours.open_now',
 'plus_code.compound_code',
 'plus_code.global_code',
 'price_level']

In [51]:
g_to_cannabis_venues.drop(columns=['business_status',
                                 'icon',
                                 'id',
                                 'photos',
                                 'place_id',
                                 'rating',
                                 'reference',
                                 'scope',
                                 'types',
                                 'user_ratings_total',
                                 'vicinity',
                                 'geometry.viewport.northeast.lat',
                                 'geometry.viewport.northeast.lng',
                                 'geometry.viewport.southwest.lat',
                                 'geometry.viewport.southwest.lng',
                                 'opening_hours.open_now',
                                 'plus_code.compound_code',
                                 'plus_code.global_code',
                                 'price_level'], inplace=True)

## Save to CSV

In [52]:
compression_opts = dict(method='zip',archive_name='data/g_to_cannabis_venues.csv')  
g_to_cannabis_venues.to_csv('data/g_to_cannabis_venues.zip', index=False, compression=compression_opts)

<a id="item7"></a>
# Pull Toronto Dessert Restaurant Data from Google Places API

In [53]:
# Request #1 criteria
radius = 25000
keyword = 'Dessert'
to_lat = 43.6532
to_long = -79.3832

In [54]:
# Request 1
url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={},{}&radius={}&keyword={}&key={}'.format(to_lat, to_long, radius, keyword,g_api_key)
results = requests.get(url).json()
# Convert venues to df
gvenues = json_normalize(results['results'])

In [55]:
# Request 2 using next page token
npt=results['next_page_token']
url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?pagetoken={}&key={}'.format(npt, g_api_key)
results2 = requests.get(url).json()
# Convert venues to df
gvenues2 = json_normalize(results2['results'])

In [56]:
# Request 3
npt=results2['next_page_token']
url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?pagetoken={}&key={}'.format(npt, g_api_key)
results3 = requests.get(url).json()
# Convert venues to df
gvenues3 = json_normalize(results3['results'])

In [57]:
# Merge the 3 DFs
g_to_dessert_venues = gvenues.append(gvenues2, ignore_index=True)
g_to_dessert_venues = g_to_dessert_venues.append(gvenues3, ignore_index=True)

In [58]:
g_to_dessert_venues.head(2)

,business_status,icon,id,name,permanently_closed,photos,place_id,price_level,rating,reference,scope,types,user_ratings_total,vicinity,geometry.location.lat,geometry.location.lng,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,plus_code.compound_code,plus_code.global_code,opening_hours.open_now
0,CLOSED_TEMPORARILY,https://maps.gstatic.com/mapfiles/place_api/ic...,af3d3db52c8066057501357a3518968beaf560e6,Roselle Desserts,True,"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJTREIwj7L1IkRDkOWEvVdY_g,2.0,4.7,ChIJTREIwj7L1IkRDkOWEvVdY_g,GOOGLE,"[food, point_of_interest, store, establishment]",690,"362 King St E, Toronto",43.653543,-79.362048,43.654853,-79.360653,43.652153,-79.363352,"MJ3Q+C5 Toronto, Ontario",87M2MJ3Q+C5,NaN
1,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,42c140d5b7b494d30160fe797943202587e1aa85,Decadent Desserts,NaN,"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJCX1hZt8wK4gRU7AV86M5PT8,NaN,4.2,ChIJCX1hZt8wK4gRU7AV86M5PT8,GOOGLE,"[bakery, food, point_of_interest, store, estab...",33,"100 King St W, Toronto",43.648559,-79.382656,43.649828,-79.380731,43.647128,-79.383431,"JJX8+CW Toronto, Ontario",87M2JJX8+CW,True


In [59]:
list(g_to_dessert_venues)

['business_status',
 'icon',
 'id',
 'name',
 'permanently_closed',
 'photos',
 'place_id',
 'price_level',
 'rating',
 'reference',
 'scope',
 'types',
 'user_ratings_total',
 'vicinity',
 'geometry.location.lat',
 'geometry.location.lng',
 'geometry.viewport.northeast.lat',
 'geometry.viewport.northeast.lng',
 'geometry.viewport.southwest.lat',
 'geometry.viewport.southwest.lng',
 'plus_code.compound_code',
 'plus_code.global_code',
 'opening_hours.open_now']

In [60]:
g_to_dessert_venues.drop(columns=['business_status',
                                 'icon',
                                 'id',
                                 'permanently_closed',
                                 'photos',
                                 'place_id',
                                 'price_level',
                                 'rating',
                                 'reference',
                                 'scope',
                                 'types',
                                 'user_ratings_total',
                                 'vicinity',
                                 'geometry.viewport.northeast.lat',
                                 'geometry.viewport.northeast.lng',
                                 'geometry.viewport.southwest.lat',
                                 'geometry.viewport.southwest.lng',
                                 'plus_code.compound_code',
                                 'plus_code.global_code',
                                 'opening_hours.open_now'], inplace=True)

## Save to CSV

In [61]:
compression_opts = dict(method='zip',archive_name='data/g_to_dessert_venues.csv')  
g_to_dessert_venues.to_csv('data/g_to_dessert_venues.zip', index=False, compression=compression_opts)

<a id="item8"></a>
# Create Toronto Map

In [133]:
# Center map based on mean of long lat of possesion charges
latitude=g_to_dessert_venues[['geometry.location.lat']].mean()
longitude=g_to_dessert_venues[['geometry.location.lng']].mean()

# Create map
to_map = folium.Map(location=[latitude, longitude], zoom_start=13, tiles='cartodbpositron')

feature_can_store = folium.FeatureGroup(name='Cannabis Store')
feature_rest = folium.FeatureGroup(name='Dessert Restaurants')

HeatMap(g_to_cannabis_venues[['geometry.location.lat','geometry.location.lng']], radius=35).add_to(feature_can_store)
                                       
for g, v in g_to_dessert_venues.iterrows():
    popup = """
    Restaurant Name : <b>%s</b><br>
    """ % (v['name'])
    folium.Marker(location=[v['geometry.location.lat'], v['geometry.location.lng']],
    radius=3,
    tooltip=popup,
    icon=folium.Icon(color='blue', icon='fa-foursquare', prefix='fa')
    ).add_to(feature_rest)

folium.CircleMarker(location=[43.6485, -79.3935], radius=20, popup='Circle 1', color='#3186cc', fill=True, fill_color='#3186cc').add_to(to_map)
folium.CircleMarker(location=[43.6485, -79.3935], radius=2, popup='Circle 1', color='#3186cc', fill=True, fill_color='#3186cc').add_to(to_map)

folium.CircleMarker(location=[43.659, -79.3816], radius=20, popup='Circle 2', color='#3186cc', fill=True, fill_color='#3186cc').add_to(to_map)
folium.CircleMarker(location=[43.659, -79.3816], radius=2, popup='Circle 2', color='#3186cc', fill=True, fill_color='#3186cc').add_to(to_map)

folium.CircleMarker(location=[43.6695, -79.39], radius=15, popup='Circle 3', color='#3186cc', fill=True, fill_color='#3186cc').add_to(to_map)
folium.CircleMarker(location=[43.6695, -79.39], radius=2, popup='Circle 3', color='#3186cc', fill=True, fill_color='#3186cc').add_to(to_map)

feature_can_store.add_to(to_map)
feature_rest.add_to(to_map)

folium.LayerControl(collapsed=False).add_to(to_map)

# Display map
to_map

In [128]:
coords_2 = (float(g_to_dessert_venues[g_to_dessert_venues['name']=='Millie Patisserie'].iloc[:,1]),float(g_to_dessert_venues[g_to_dessert_venues['name']=='Millie Patisserie'].iloc[:,2]))

coords_2

(43.6465049, -79.39427839999999)

In [134]:
# Determine distance of the most dense possesion areas in red to the closest dessert restaurant
import geopy.distance

coords_1 = (43.6485, -79.3935)
coords_2 = (float(g_to_dessert_venues[g_to_dessert_venues['name']=='Millie Patisserie'].iloc[:,1]),float(g_to_dessert_venues[g_to_dessert_venues['name']=='Millie Patisserie'].iloc[:,2]))
to_distance1 = geopy.distance.distance(coords_1, coords_2).km

coords_1 = (43.659, -79.3816)
coords_2 = (float(g_to_dessert_venues[g_to_dessert_venues['name']=='Butter Baker'].iloc[:,1]),float(g_to_dessert_venues[g_to_dessert_venues['name']=='Butter Baker'].iloc[:,2]))
to_distance2 = geopy.distance.distance(coords_1, coords_2).km

coords_1 = (43.6695, -79.39)
coords_2 = (float(g_to_dessert_venues[g_to_dessert_venues['name']=='Dessert Lady Cafe & Bakery'].iloc[:,1]),float(g_to_dessert_venues[g_to_dessert_venues['name']=='Dessert Lady Cafe & Bakery'].iloc[:,2]))
to_distance3 = geopy.distance.distance(coords_1, coords_2).km

print('Circle 1 Distance: {:.2f}.km\nCircle 2 Distance: {:.2f}.km\nCircle 3 Distance: {:.2f}.km'.format(munchie_distance1, munchie_distance2, munchie_distance3))

Circle 1 Distance: 0.23.km
Circle 2 Distance: 0.43.km
Circle 3 Distance: 0.09.km


In [135]:
to_average_distance = (to_distance1+to_distance2+to_distance3)/3
to_average_distance

0.25072190238121417

<a id="item9"></a>
# Difference for Chicago Vs Toronto

In [138]:
diff_distance=chi_average_distance-to_average_distance
diff_distance

5.011653047646886